#1. Preparation

Runtime => GPU

In [1]:
import pandas as pd
import os
import cv2

In [42]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [87]:
COLAB_DARKNET_PATH = '/gdrive/MyDrive/Darknet'

DATA_PATH = '/Train'
YOLO_IMAGE_PATH = COLAB_DARKNET_PATH + DATA_PATH
YOLO_FORMAT_PATH = COLAB_DARKNET_PATH + DATA_PATH

test_percentage = 0.2

!mkdir backup
!mkdir Train

mkdir: cannot create directory ‘backup’: File exists
mkdir: cannot create directory ‘Train’: File exists


In [6]:
# # Train.csv
# from google.colab import files
# files.upload()

Saving Train.csv to Train.csv


{'Train.csv': b'Width,Height,Roi.X1,Roi.Y1,Roi.X2,Roi.Y2,ClassId,Path\n27,26,5,5,22,20,20,Train/20/00020_00000_00000.png\n28,27,5,6,23,22,20,Train/20/00020_00000_00001.png\n29,26,6,5,24,21,20,Train/20/00020_00000_00002.png\n28,27,5,6,23,22,20,Train/20/00020_00000_00003.png\n28,26,5,5,23,21,20,Train/20/00020_00000_00004.png\n31,27,6,5,26,22,20,Train/20/00020_00000_00005.png\n31,28,6,6,26,23,20,Train/20/00020_00000_00006.png\n31,28,6,6,26,23,20,Train/20/00020_00000_00007.png\n31,29,5,6,26,24,20,Train/20/00020_00000_00008.png\n34,32,6,6,29,26,20,Train/20/00020_00000_00009.png\n36,33,5,6,31,28,20,Train/20/00020_00000_00010.png\n37,34,5,6,32,29,20,Train/20/00020_00000_00011.png\n38,34,5,6,32,29,20,Train/20/00020_00000_00012.png\n40,34,6,6,34,29,20,Train/20/00020_00000_00013.png\n39,34,5,5,34,29,20,Train/20/00020_00000_00014.png\n42,36,6,5,37,31,20,Train/20/00020_00000_00015.png\n45,39,6,5,40,34,20,Train/20/00020_00000_00016.png\n47,42,5,5,41,36,20,Train/20/00020_00000_00017.png\n50,45,5,5,4

In [44]:
!cp -r /gdrive/MyDrive/Darknet/Train.zip .

In [63]:
!unzip Train.zip

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: Train/5/00005_00053_00014.png  
  inflating: Train/5/00005_00053_00015.png  
  inflating: Train/5/00005_00053_00016.png  
  inflating: Train/5/00005_00053_00017.png  
  inflating: Train/5/00005_00053_00018.png  
  inflating: Train/5/00005_00053_00019.png  
  inflating: Train/5/00005_00053_00020.png  
  inflating: Train/5/00005_00053_00021.png  
  inflating: Train/5/00005_00053_00022.png  
  inflating: Train/5/00005_00053_00023.png  
  inflating: Train/5/00005_00053_00024.png  
  inflating: Train/5/00005_00053_00025.png  
  inflating: Train/5/00005_00053_00026.png  
  inflating: Train/5/00005_00053_00027.png  
  inflating: Train/5/00005_00053_00028.png  
  inflating: Train/5/00005_00053_00029.png  
  inflating: Train/5/00005_00054_00000.png  
  inflating: Train/5/00005_00054_00001.png  
  inflating: Train/5/00005_00054_00002.png  
  inflating: Train/5/00005_00054_00003.png  
  inflating: Train/5/00005_00054_00004.png  
  inflating: Train/

In [45]:
!ls -al

total 238528
drwxr-xr-x 1 root root      4096 May 18 11:33 .
drwxr-xr-x 1 root root      4096 May 18 11:29 ..
drwxr-xr-x 2 root root      4096 May 18 11:15 backup
-rw-r--r-- 1 root root        35 May 18 11:26 classes.names
drwxr-xr-x 4 root root      4096 May 16 13:23 .config
drwxr-xr-x 1 root root      4096 May 16 13:24 sample_data
drwxr-xr-x 2 root root      4096 May 18 11:15 Train
-rw-r--r-- 1 root root   1940896 May 18 11:17 Train.csv
-rw------- 1 root root 242277449 May 18 11:33 Train.zip


In [50]:
annotation = pd.read_csv("Train.csv")


In [51]:
annotation.head()

,Width,Height,Roi.X1,Roi.Y1,Roi.X2,Roi.Y2,ClassId,Path
0,27,26,5,5,22,20,20,Train/20/00020_00000_00000.png
1,28,27,5,6,23,22,20,Train/20/00020_00000_00001.png
2,29,26,6,5,24,21,20,Train/20/00020_00000_00002.png
3,28,27,5,6,23,22,20,Train/20/00020_00000_00003.png
4,28,26,5,5,23,21,20,Train/20/00020_00000_00004.png


In [52]:
Prohibitory = [0,1,2,3,4,5,6,7,8,10,15,16]
Danger = [11,18,19,20,21,22,23,24,25,26,27,28,29,30,31]
Mandatory = [33,34,35,36,37,38,39,40]
Other = [6,12,13,14,17,32,41,42]

Categories = ['Prohibitory', 'Danger', 'Mandatory', 'Other']

In [53]:
annotation["CategoryId"] = 0
annotation.loc[annotation['ClassId'].isin(Prohibitory), "CategoryId"] = 0
annotation.loc[annotation['ClassId'].isin(Danger), "CategoryId"] = 1
annotation.loc[annotation['ClassId'].isin(Mandatory), "CategoryId"] = 2
annotation.loc[annotation['ClassId'].isin(Other), "CategoryId"] = 3

In [54]:
annotation['centerX'] = (annotation['Roi.X2']+annotation['Roi.X1']) /2 / annotation['Width']
annotation['centerY'] = (annotation['Roi.Y2']+annotation['Roi.Y1']) /2 / annotation['Height']
annotation['width'] = (annotation['Roi.X2']-annotation['Roi.X1']) / annotation['Width']
annotation['height'] = (annotation['Roi.Y2']-annotation['Roi.Y1']) / annotation['Height']

In [55]:
annotation = annotation.drop(['Width', 'Height', 'Roi.X1', 'Roi.Y1', 'Roi.X2', 'Roi.Y2', 'ClassId' ], axis='columns')

In [56]:
for index, data in annotation.iterrows():
  print(index, data['Path'])

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
34209 Train/35/00035_00000_00010.png
34210 Train/35/00035_00000_00011.png
34211 Train/35/00035_00000_00012.png
34212 Train/35/00035_00000_00013.png
34213 Train/35/00035_00000_00014.png
34214 Train/35/00035_00000_00015.png
34215 Train/35/00035_00000_00016.png
34216 Train/35/00035_00000_00017.png
34217 Train/35/00035_00000_00018.png
34218 Train/35/00035_00000_00019.png
34219 Train/35/00035_00000_00020.png
34220 Train/35/00035_00000_00021.png
34221 Train/35/00035_00000_00022.png
34222 Train/35/00035_00000_00023.png
34223 Train/35/00035_00000_00024.png
34224 Train/35/00035_00000_00025.png
34225 Train/35/00035_00000_00026.png
34226 Train/35/00035_00000_00027.png
34227 Train/35/00035_00000_00028.png
34228 Train/35/00035_00000_00029.png
34229 Train/35/00035_00001_00000.png
34230 Train/35/00035_00001_00001.png
34231 Train/35/00035_00001_00002.png
34232 Train/35/00035_00001_00003.png
34233 Train/35/00035_00001_00004.png
34234 Train/35/00035_00001_00005.png
34

In [57]:
with open( 'classes.names', 'w') as names:
    for category in Categories:
      line = category + '\n'
      names.write(line)
    print('[classes.names] is created')

[classes.names] is created


In [58]:
!ls -al

total 238528
drwxr-xr-x 1 root root      4096 May 18 11:33 .
drwxr-xr-x 1 root root      4096 May 18 11:29 ..
drwxr-xr-x 2 root root      4096 May 18 11:15 backup
-rw-r--r-- 1 root root        35 May 18 11:35 classes.names
drwxr-xr-x 4 root root      4096 May 16 13:23 .config
drwxr-xr-x 1 root root      4096 May 16 13:24 sample_data
drwxr-xr-x 2 root root      4096 May 18 11:15 Train
-rw-r--r-- 1 root root   1940896 May 18 11:17 Train.csv
-rw------- 1 root root 242277449 May 18 11:33 Train.zip


In [59]:
# Download classes.names 
from google.colab import files
files.download('classes.names')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [68]:
with open('traffic.data', 'w') as data:
    data.write('classes = ' + str(len(Categories)) + '\n')
    data.write('train = ' + 'train.txt' + '\n')
    data.write('valid = ' + 'test.txt' + '\n')
    data.write('names = ' + 'classes.names' + '\n')
    data.write('backup = backup')
    print('[custom_data.data] is created')

[custom_data.data] is created


In [71]:
# Download classes.names 
from google.colab import files
files.download('traffic.data')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [70]:
!mkdir backup

mkdir: cannot create directory ‘backup’: File exists


In [72]:
!ls -al

total 238536
drwxr-xr-x  1 root root      4096 May 18 11:43 .
drwxr-xr-x  1 root root      4096 May 18 11:29 ..
drwxr-xr-x  2 root root      4096 May 18 11:15 backup
drwxr-xr-x  2 root root      4096 May 18 11:43 BACKUP
-rw-r--r--  1 root root        35 May 18 11:35 classes.names
drwxr-xr-x  4 root root      4096 May 16 13:23 .config
drwxr-xr-x  1 root root      4096 May 16 13:24 sample_data
-rw-r--r--  1 root root        84 May 18 11:43 traffic.data
drwxr-xr-x 45 root root      4096 May 18 11:38 Train
-rw-r--r--  1 root root   1940896 May 18 11:17 Train.csv
-rw-------  1 root root 242277449 May 18 11:33 Train.zip


In [79]:
annotation.head()

,Path,CategoryId,centerX,centerY,width,height
0,Train/20/00020_00000_00000.png,1,0.500000,0.480769,0.629630,0.576923
1,Train/20/00020_00000_00001.png,1,0.500000,0.518519,0.642857,0.592593
2,Train/20/00020_00000_00002.png,1,0.517241,0.500000,0.620690,0.615385
3,Train/20/00020_00000_00003.png,1,0.500000,0.518519,0.642857,0.592593
4,Train/20/00020_00000_00004.png,1,0.500000,0.500000,0.642857,0.615385


In [84]:
paths = []
for index, data in annotation.iterrows():
  txtFile = data['Path'].split('.')[0] + '.txt'
  line = str(data['CategoryId']) + ' ' + str(data['centerX']) + ' ' + str(data['centerY']) + ' ' + str(data['width']) + ' ' + str(data['height']) + '\n'
  #print(line)
  with open(txtFile, 'w') as txt:
      txt.write(line)
  # print(index, data['Path'])
  imagePath = data['Path'] + '\n'
  paths.append(imagePath )
  # print(index, txtFile, line)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
36709 Train/38/00038_00023_00020.txt 2 0.5094339622641509 0.5 0.7924528301886793 0.8113207547169812

36710 Train/38/00038_00023_00021.txt 2 0.5 0.5 0.7857142857142857 0.8181818181818182

36711 Train/38/00038_00023_00022.txt 2 0.5 0.5 0.8032786885245902 0.8333333333333334

36712 Train/38/00038_00023_00023.txt 2 0.5 0.4921875 0.8125 0.828125

36713 Train/38/00038_00023_00024.txt 2 0.49264705882352944 0.5 0.8088235294117647 0.8260869565217391

36714 Train/38/00038_00023_00025.txt 2 0.5066666666666667 0.5066666666666667 0.8266666666666667 0.8266666666666667

36715 Train/38/00038_00023_00026.txt 2 0.5059523809523809 0.5058823529411764 0.8214285714285714 0.8235294117647058

36716 Train/38/00038_00023_00027.txt 2 0.5 0.5 0.8333333333333334 0.8367346938775511

36717 Train/38/00038_00023_00028.txt 2 0.5 0.5044642857142857 0.8333333333333334 0.8303571428571429

36718 Train/38/00038_00023_00029.txt 2 0.5 0.5036496350364964 0.8307692307692308 0.8321167883211679


In [ ]:
paths

In [85]:
!ls - al Train/42/

ls: cannot access '-': No such file or directory
ls: cannot access 'al': No such file or directory
Train/42/:
00042_00000_00000.png  00042_00002_00020.png  00042_00005_00010.png
00042_00000_00000.txt  00042_00002_00020.txt  00042_00005_00010.txt
00042_00000_00001.png  00042_00002_00021.png  00042_00005_00011.png
00042_00000_00001.txt  00042_00002_00021.txt  00042_00005_00011.txt
00042_00000_00002.png  00042_00002_00022.png  00042_00005_00012.png
00042_00000_00002.txt  00042_00002_00022.txt  00042_00005_00012.txt
00042_00000_00003.png  00042_00002_00023.png  00042_00005_00013.png
00042_00000_00003.txt  00042_00002_00023.txt  00042_00005_00013.txt
00042_00000_00004.png  00042_00002_00024.png  00042_00005_00014.png
00042_00000_00004.txt  00042_00002_00024.txt  00042_00005_00014.txt
00042_00000_00005.png  00042_00002_00025.png  00042_00005_00015.png
00042_00000_00005.txt  00042_00002_00025.txt  00042_00005_00015.txt
00042_00000_00006.png  00042_00002_00026.png  00042_00005_00016.png
00042_

In [86]:
# Download classes.names 
from google.colab import files
files.download('Train/42/00042_00002_00019.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [88]:
num_test = int(len(paths) * test_percentage)
print(num_test)

7841


In [89]:
paths_test = paths[:num_test]
paths_train = paths[num_test:]

In [90]:
with open('train.txt', 'w') as train_txt:
  for path in paths_train:
    train_txt.write(path)
  print('train.txt is created')

train.txt is created


In [91]:
with open('test.txt', 'w') as test_txt:
  for path in paths_test:
    test_txt.write(path)
  print('test_txt is created')

test_txt is created


In [96]:
from google.colab import files
files.download('train.txt')
files.download('test.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [92]:
!ls -al

total 239712
drwxr-xr-x  1 root root      4096 May 18 12:08 .
drwxr-xr-x  1 root root      4096 May 18 11:29 ..
drwxr-xr-x  2 root root      4096 May 18 11:15 backup
drwxr-xr-x  2 root root      4096 May 18 11:43 BACKUP
-rw-r--r--  1 root root        35 May 18 11:35 classes.names
drwxr-xr-x  4 root root      4096 May 16 13:23 .config
drwxr-xr-x  1 root root      4096 May 16 13:24 sample_data
-rw-r--r--  1 root root    235590 May 18 12:08 test.txt
-rw-r--r--  1 root root        84 May 18 11:56 traffic.data
drwxr-xr-x 45 root root      4096 May 18 11:38 Train
-rw-r--r--  1 root root   1940896 May 18 11:17 Train.csv
-rw-r--r--  1 root root    965219 May 18 12:07 train.txt
-rw-------  1 root root 242277449 May 18 11:33 Train.zip


In [98]:
!cp -r /gdrive/MyDrive/Darknet/Custom/darknet .

In [99]:
!cp -r /gdrive/MyDrive/Darknet/Custom/Traffic/traffic-train-yolo.cfg .

In [100]:
!cp -r /gdrive/MyDrive/Darknet/Custom/darknet53.conv.74 .

In [101]:
!ls -al

total 401544
drwxr-xr-x  1 root root      4096 May 18 12:19 .
drwxr-xr-x  1 root root      4096 May 18 11:29 ..
drwxr-xr-x  2 root root      4096 May 18 11:15 backup
drwxr-xr-x  2 root root      4096 May 18 11:43 BACKUP
-rw-r--r--  1 root root        35 May 18 11:35 classes.names
drwxr-xr-x  4 root root      4096 May 16 13:23 .config
-rw-------  1 root root   3213624 May 18 12:18 darknet
-rw-------  1 root root 162482580 May 18 12:19 darknet53.conv.74
drwxr-xr-x  1 root root      4096 May 16 13:24 sample_data
-rw-r--r--  1 root root    235590 May 18 12:08 test.txt
-rw-r--r--  1 root root        84 May 18 11:56 traffic.data
-rw-------  1 root root      8328 May 18 12:19 traffic-train-yolo.cfg
drwxr-xr-x 45 root root      4096 May 18 11:38 Train
-rw-r--r--  1 root root   1940896 May 18 11:17 Train.csv
-rw-r--r--  1 root root    965219 May 18 12:07 train.txt
-rw-------  1 root root 242277449 May 18 11:33 Train.zip


In [103]:
!chmod +x ./darknet

In [ ]:
# Train Model
!./darknet detector train traffic.data traffic-train-yolo.cfg darknet53.conv.74 -dont_show

traffic-train-yolo
layer     filters    size              input                output
   0 conv     32  3 x 3 / 1   416 x 416 x   3   ->   416 x 416 x  32 0.299 BF
   1 conv     64  3 x 3 / 2   416 x 416 x  32   ->   208 x 208 x  64 1.595 BF
   2 conv     32  1 x 1 / 1   208 x 208 x  64   ->   208 x 208 x  32 0.177 BF
   3 conv     64  3 x 3 / 1   208 x 208 x  32   ->   208 x 208 x  64 1.595 BF
   4 Shortcut Layer: 1
   5 conv    128  3 x 3 / 2   208 x 208 x  64   ->   104 x 104 x 128 1.595 BF
   6 conv     64  1 x 1 / 1   104 x 104 x 128   ->   104 x 104 x  64 0.177 BF
   7 conv    128  3 x 3 / 1   104 x 104 x  64   ->   104 x 104 x 128 1.595 BF
   8 Shortcut Layer: 5
   9 conv     64  1 x 1 / 1   104 x 104 x 128   ->   104 x 104 x  64 0.177 BF
  10 conv    128  3 x 3 / 1   104 x 104 x  64   ->   104 x 104 x 128 1.595 BF
  11 Shortcut Layer: 8
  12 conv    256  3 x 3 / 2   104 x 104 x 128   ->    52 x  52 x 256 1.595 BF
  13 conv    128  1 x 1 / 1    52 x  52 x 256   ->    52 x  52 x 